In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Exploration du fichier des vitesses

Ce fichier contient les vitesses maximales autorisées sur les tronçons du réseau ferré national.

In [2]:
speeds = gpd.read_file("../data/raw/vitesse-maximale-nominale-sur-ligne.geojson")
print(speeds.shape)
speeds.head()

(2529, 19)


,code_ligne,lib_ligne,v_max,rg_troncon,pkd,pkf,idgaia,x_d_l93,y_d_l93,x_f_l93,y_f_l93,x_d_wgs84,y_d_wgs84,x_f_wgs84,y_f_wgs84,c_geo_d,c_geo_f,geo_point_2d,geometry
0,242000,Ligne de Creil à Jeumont,140,1,222+152,224+841,77986fb2-e28c-11e8-92ff-01b064e0362d,7.641287e+05,7.016495e+06,7.658409e+05,7.018610e+06,3.897931,50.243157,3.922249,50.261966,"{ ""lon"": 3.897931336245875, ""lat"": 50.24315723...","{ ""lon"": 3.9222490335686517, ""lat"": 50.2619658...","{ ""lon"": 3.9108305177413909, ""lat"": 50.2517217...","LINESTRING (3.89793 50.24316, 3.8982 50.24339,..."
1,366000,Ligne de Mantes-la-Jolie à Cherbourg,200,1,204+738,238+908,77a4fa62-e28c-11e8-92ff-01b064e0362d,4.856911e+05,6.893278e+06,4.559102e+05,6.902591e+06,0.064633,49.103467,-0.348220,49.176484,"{ ""lon"": 0.064633482429558339, ""lat"": 49.10346...","{ ""lon"": -0.34821966895941142, ""lat"": 49.17648...","{ ""lon"": -0.14445964061185351, ""lat"": 49.10945...","LINESTRING (0.06463 49.10347, 0.06453 49.10343..."
2,995000,Ligne de Bastia à Ajaccio (ligne centrale),60,1,091+100,092+550,77909e06-e28c-11e8-92ff-01b064e0362d,1.209888e+06,6.140752e+06,1.210450e+06,6.139784e+06,9.167456,42.187707,9.173311,42.178645,"{ ""lon"": 9.1674555515289811, ""lat"": 42.1877074...","{ ""lon"": 9.1733106648204128, ""lat"": 42.1786453...","{ ""lon"": 9.1708395291329747, ""lat"": 42.1833669...","LINESTRING (9.16746 42.18771, 9.1676 42.18756,..."
3,995000,Ligne de Bastia à Ajaccio (ligne centrale),70,1,028+280,031+800,778e2f14-e28c-11e8-92ff-01b064e0362d,1.224691e+06,6.177971e+06,1.221449e+06,6.177839e+06,9.382049,42.510609,9.342656,42.511779,"{ ""lon"": 9.3820493826610285, ""lat"": 42.5106094...","{ ""lon"": 9.3426559493521957, ""lat"": 42.5117794...","{ ""lon"": 9.3623621168976765, ""lat"": 42.5093457...","LINESTRING (9.38205 42.51061, 9.3752 42.50851,..."
4,890306,Raccordement de Culoz,60,1,000+000,000+606,779306f0-e28c-11e8-92ff-01b064e0362d,9.162122e+05,6.531116e+06,9.160776e+05,6.530596e+06,5.786444,45.845357,5.784475,45.840723,"{ ""lon"": 5.7864437996495486, ""lat"": 45.8453573...","{ ""lon"": 5.7844746068288542, ""lat"": 45.8407233...","{ ""lon"": 5.7845964512461716, ""lat"": 45.8432853...","LINESTRING (5.78644 45.84536, 5.78532 45.84474..."


## Réduction des données

Les colonnes ```"rg_troncon"```, ```"pkd"```, ```"pkf"```, ```"geo_point_2d"```, ```"geometry"``` sont déjà présentes dans le fichier ```"formes-des-lignes-du-rfn.geojson"```, donc elles sont redondantes. On garde néanmoins la colonne ```"code_ligne"``` comme clé pour le merging plus tard.

Les colonnes ```'x_d_l93','y_d_l93','x_f_l93','y_f_l93','x_d_wgs84','y_d_wgs84','x_f_wgs84','y_f_wgs84','c_geo_d','c_geo_f'``` sont des coordonnées pour différentes projections cartographiques. Elles ne sont donc pas pertinentes.

<https://fr.wikipedia.org/wiki/Projection_conique_conforme_de_Lambert>

<https://fr.wikipedia.org/wiki/WGS_84#:~:text=WGS%2084%20(World%20Geodetic%20System,g%C3%A9o%C3%AFde%20(EGM96%20ou%20suivant).>

```"idgaia"``` est un Id qui n'apparait nulle part ailleurs dans les données, on n'en a pas d'utilité, d'autant plus qu'on a déjà un identifiant pour les tronçons, avec ```"code_ligne"```.

In [3]:
speeds.dtypes

code_ligne        object
lib_ligne         object
v_max             object
rg_troncon         int32
pkd               object
pkf               object
idgaia            object
x_d_l93          float64
y_d_l93          float64
x_f_l93          float64
y_f_l93          float64
x_d_wgs84        float64
y_d_wgs84        float64
x_f_wgs84        float64
y_f_wgs84        float64
c_geo_d           object
c_geo_f           object
geo_point_2d      object
geometry        geometry
dtype: object

### ```"v_max"```

In [4]:
speeds["v_max"].unique()

array(['140', '200', '60', '70', '100', '155', '120', '90', '30', None,
       '150', '50', '230', '130', '160', '45', '40', '20', '80', '110',
       '95', '320', '220', '75', '270', '115', '105', '55', '12', '125',
       '85', '300', '35', '180', '170', '350', '15', '135', '65', '210',
       '190', '145', '10'], dtype=object)

In [5]:
speeds[speeds["v_max"].isna()].head()

,code_ligne,lib_ligne,v_max,rg_troncon,pkd,pkf,idgaia,x_d_l93,y_d_l93,x_f_l93,y_f_l93,x_d_wgs84,y_d_wgs84,x_f_wgs84,y_f_wgs84,c_geo_d,c_geo_f,geo_point_2d,geometry
15,912000,Ligne de Livron à Aspres-sur-Buëch,None,1,109+283,109+452,c7cb42b0-37ff-11ec-80ff-01206fb51c27,918854.9903,6.383857e+06,918902.7226,6.383695e+06,5.754304,44.519420,5.754833,44.517950,"{ ""lon"": 5.7543039196877448, ""lat"": 44.5194196...","{ ""lon"": 5.7548331700559316, ""lat"": 44.5179497...","{ ""lon"": 5.7546112441567763, ""lat"": 44.5187008...","LINESTRING (5.7543 44.51942, 5.75433 44.51938,..."
24,314611,Voie-mère de Calais,None,1,000+000,005+396,7f450962-116d-11ec-84ff-017c6fb51c27,619996.2250,7.096184e+06,620673.8615,7.097991e+06,1.863757,50.956438,1.873014,50.972733,"{ ""lon"": 1.8637574136853148, ""lat"": 50.9564378...","{ ""lon"": 1.8730138604133666, ""lat"": 50.9727325...","{ ""lon"": 1.8837870334451934, ""lat"": 50.9674002...","LINESTRING (1.86376 50.95644, 1.86418 50.9568,..."
30,990000,Ligne de la grande ceinture de Paris,None,4,096+775,109+226,a7f2caa0-cde7-11eb-94ff-0110b2273146,645836.5264,6.847994e+06,643452.4926,6.849165e+06,2.263595,48.729803,2.231029,48.740123,"{ ""lon"": 2.263595187347017, ""lat"": 48.72980326...","{ ""lon"": 2.2310290244226727, ""lat"": 48.7401226...","{ ""lon"": 2.2864731154259883, ""lat"": 48.7079907...","MULTILINESTRING ((2.2636 48.7298, 2.26277 48.7..."
49,985000,Ligne de Choisy-le-Roi à Massy-Verrières,None,1,013+353,025+435,77981a42-e28c-11e8-92ff-01b064e0362d,649456.8177,6.849975e+06,649456.8177,6.849975e+06,2.312580,48.747909,2.312580,48.747909,"{ ""lon"": 2.3125795585876001, ""lat"": 48.7479094...","{ ""lon"": 2.3125795585876001, ""lat"": 48.7479094...","{ ""lon"": 2.3255000606397984, ""lat"": 48.7425218...","MULTILINESTRING ((2.31258 48.74791, 2.3119 48...."
52,830000,Ligne de Paris-Lyon à Marseille-St-Charles,None,1,859+371,860+844,779ddc14-e28c-11e8-92ff-01b064e0362d,894149.2162,6.250061e+06,894150.4737,6.248602e+06,5.392582,43.323227,5.392053,43.310107,"{ ""lon"": 5.3925823966289359, ""lat"": 43.3232273...","{ ""lon"": 5.392053324276378, ""lat"": 43.31010692...","{ ""lon"": 5.3930299247111879, ""lat"": 43.3165904...","LINESTRING (5.39258 43.32323, 5.39262 43.32311..."


On suppose que s'il n'y a pas de limitation de vitesse, alors la vitesse maximale sur le tronçon vaut la valeur maximale de la colonne ```"v_max"```, soit 350 km/h.

In [6]:
def process_speeds(speeds_df, ignore_na=True):
    relevant_columns = ["code_ligne", "lib_ligne", "v_max","geometry","pkd","pkf"] # On garde les colonnes qui peuvent être des clés primaires (voir 1_shapes.ipynb)
    speeds_df_processed = speeds_df[relevant_columns].copy() # On garde une copie pour éviter de modifier l'original
    speeds_df_processed["v_max"] = speeds_df_processed["v_max"].astype("Int64") # v_max est par défaut un object, on le convertit en int64
    # On convertit la vitesse max en entier nullable (.astype(int) ne fonctionne que pour les entiers non-nullables, 
    # voir https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int)
    if ignore_na:
        speeds_df_processed = speeds_df_processed[~speeds_df_processed["v_max"].isna()]
        # Si on ignore les NaN, on supprime les lignes qui en contiennent
    else:
        max_speed = speeds_df_processed["v_max"].max()
        speeds_df_processed["v_max"] = speeds_df_processed["v_max"].fillna(max_speed)
        # Si on ne les ignore pas, on remplace les NaN par la vitesse max de la colonne
    speeds_df_processed = speeds_df_processed.reset_index(drop=True)
    # On renomme les colonnes pkd et pkf en pk_debut_r et pk_fin_r pour être cohérent avec le fichier shapes
    speeds_df_processed = speeds_df_processed.rename(columns={"pkd":"pk_debut_r","pkf":"pk_fin_r"})
    return speeds_df_processed

speeds_processed = process_speeds(speeds, ignore_na=True)
print(speeds_processed.shape)
speeds_processed.head()

(2165, 6)


,code_ligne,lib_ligne,v_max,geometry,pk_debut_r,pk_fin_r
0,242000,Ligne de Creil à Jeumont,140,"LINESTRING (3.89793 50.24316, 3.8982 50.24339,...",222+152,224+841
1,366000,Ligne de Mantes-la-Jolie à Cherbourg,200,"LINESTRING (0.06463 49.10347, 0.06453 49.10343...",204+738,238+908
2,995000,Ligne de Bastia à Ajaccio (ligne centrale),60,"LINESTRING (9.16746 42.18771, 9.1676 42.18756,...",091+100,092+550
3,995000,Ligne de Bastia à Ajaccio (ligne centrale),70,"LINESTRING (9.38205 42.51061, 9.3752 42.50851,...",028+280,031+800
4,890306,Raccordement de Culoz,60,"LINESTRING (5.78644 45.84536, 5.78532 45.84474...",000+000,000+606


In [7]:
speeds_processed.dtypes

code_ligne      object
lib_ligne       object
v_max            Int64
geometry      geometry
pk_debut_r      object
pk_fin_r        object
dtype: object

## Bonus: Histogramme générique

In [8]:
def generate_histogram(speeds_df):
    fig = px.histogram(speeds_df, x="v_max", nbins=50)
    return fig

generate_histogram(speeds_processed)

In [9]:
generate_histogram(process_speeds(speeds, ignore_na=False))

Lorsque "v_max" vaut ```NA```, alors c'est qu'il n'y a pas de limitations de vitesse et que donc la vitesse maximale est de 350, On a une anomalie dans l'histogramme.

De plus, en analysant les lignes où la limitation de vitesse est haute, il est spécifié s'agit de LGV (lignes à grande vitesse). Or, dans les lignes où "v_max" vaut ```NA```, cette précision n'est pas présente. On ne peut donc pas tirer de conclusions sur les limitations de vitesses qui ne sont pas présentes dans le dataset.
Il vaut donc mieux garder les valeurs ```NA``` pour la vitesse maximale.